## SQLAlchemy ORM笔记

### 连接

In [1]:
from sqlalchemy import create_engine
engine = create_engine('mysql+pymysql://root:@localhost/test', echo=True)

### 声明一个Mapping

In [2]:
# declarative base类用于映射数据库表和Python的实体类，同样作为实体类的公共父类，通常一个应用程序只应有一个declarative base实例
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [3]:
# 定义t_user表的实体类
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 't_user'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(32))
    fullname = Column(String(200))
    password = Column(String(32))
    
    def __repr__(self):
        return "<User (id=%d, name='%s', fullname='%s', password='%s')" %(
            self.id, self.name, self.fullname, self.password)

In [4]:
# 查看User实体类对应的数据库表信息
User.__table__

Table('t_user', MetaData(bind=None), Column('id', Integer(), table=<t_user>, primary_key=True, nullable=False), Column('name', String(length=32), table=<t_user>), Column('fullname', String(length=200), table=<t_user>), Column('password', String(length=32), table=<t_user>), schema=None)

In [5]:
# 根据实体类创建数据库表
Base.metadata.create_all(engine)

2018-03-22 06:38:46,833 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-03-22 06:38:46,835 INFO sqlalchemy.engine.base.Engine {}
2018-03-22 06:38:46,840 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-03-22 06:38:46,842 INFO sqlalchemy.engine.base.Engine {}
2018-03-22 06:38:46,845 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-03-22 06:38:46,847 INFO sqlalchemy.engine.base.Engine {}
2018-03-22 06:38:46,851 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-03-22 06:38:46,853 INFO sqlalchemy.engine.base.Engine {}
2018-03-22 06:38:46,856 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-03-22 06:38:46,858 INFO sqlalchemy.engine.base.Engine {}
2018-03-22 06:38:46,863 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2018-03-22 06

In [6]:
# 创建一个User实例
ed_user = User(name='ed', fullname='Ed Jones', password='123456')
print(ed_user.id)

None


### 创建一个会话

In [7]:
# 使用我们的数据库引擎创建一个会话容器
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [8]:
# 在容器中创建一个会话对象
session = Session()

### 增加和更新对象

In [9]:
# 将新对象持久化到数据库
session.add(ed_user)
session.commit()

2018-03-22 06:38:46,952 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-22 06:38:46,956 INFO sqlalchemy.engine.base.Engine INSERT INTO t_user (name, fullname, password) VALUES (%(name)s, %(fullname)s, %(password)s)
2018-03-22 06:38:46,958 INFO sqlalchemy.engine.base.Engine {'name': 'ed', 'fullname': 'Ed Jones', 'password': '123456'}
2018-03-22 06:38:46,961 INFO sqlalchemy.engine.base.Engine COMMIT


In [10]:
# 查询看结果
session.query(User).filter_by(name='ed').first()

2018-03-22 06:38:46,973 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-22 06:38:46,979 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.name = %(name_1)s 
 LIMIT %(param_1)s
2018-03-22 06:38:46,980 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed', 'param_1': 1}


<User (id=1, name='ed', fullname='Ed Jones', password='123456')

In [11]:
# 批量添加数据
session.add_all([
    User(name='john', fullname='John Doe', password='xxxxxx'),
    User(name='mike', fullname='Micheal Jackson', password='yyyyyy'),
    User(name='sofie', fullname='Sofie Moore', password='zzzzzz')
])

In [12]:
# 修改ed的密码
ed_user.password = '654321'

In [13]:
# 查询会话的脏数据
session.dirty

IdentitySet([<User (id=1, name='ed', fullname='Ed Jones', password='654321')])

In [14]:
session.commit()

2018-03-22 06:38:47,028 INFO sqlalchemy.engine.base.Engine UPDATE t_user SET password=%(password)s WHERE t_user.id = %(t_user_id)s
2018-03-22 06:38:47,032 INFO sqlalchemy.engine.base.Engine {'password': '654321', 't_user_id': 1}
2018-03-22 06:38:47,036 INFO sqlalchemy.engine.base.Engine INSERT INTO t_user (name, fullname, password) VALUES (%(name)s, %(fullname)s, %(password)s)
2018-03-22 06:38:47,038 INFO sqlalchemy.engine.base.Engine {'name': 'john', 'fullname': 'John Doe', 'password': 'xxxxxx'}
2018-03-22 06:38:47,041 INFO sqlalchemy.engine.base.Engine INSERT INTO t_user (name, fullname, password) VALUES (%(name)s, %(fullname)s, %(password)s)
2018-03-22 06:38:47,044 INFO sqlalchemy.engine.base.Engine {'name': 'mike', 'fullname': 'Micheal Jackson', 'password': 'yyyyyy'}
2018-03-22 06:38:47,047 INFO sqlalchemy.engine.base.Engine INSERT INTO t_user (name, fullname, password) VALUES (%(name)s, %(fullname)s, %(password)s)
2018-03-22 06:38:47,050 INFO sqlalchemy.engine.base.Engine {'name':

### 事务回滚

In [15]:
# 添加一条主键冲突的记录，commit()发生异常，捕获异常并回滚
try :
    ed_user.name='edwartz'
    session.add(User(id=1, name='wrong'))
    session.commit()
except:
    session.rollback()

2018-03-22 06:38:47,070 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-22 06:38:47,074 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.id = %(param_1)s
2018-03-22 06:38:47,076 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2018-03-22 06:38:47,079 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [16]:
# 验证ed_user的name字段并未改变
ed_user.name

2018-03-22 06:38:47,090 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-22 06:38:47,093 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.id = %(param_1)s
2018-03-22 06:38:47,096 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


'ed'

### 查询

In [17]:
# 倒序查询user表
from sqlalchemy.sql import desc
for row in session.query(User).order_by(desc(User.name)):
    print(row)

2018-03-22 06:38:47,112 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user ORDER BY t_user.name DESC
2018-03-22 06:38:47,118 INFO sqlalchemy.engine.base.Engine {}
<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz')
<User (id=3, name='mike', fullname='Micheal Jackson', password='yyyyyy')
<User (id=2, name='john', fullname='John Doe', password='xxxxxx')
<User (id=1, name='ed', fullname='Ed Jones', password='654321')


In [18]:
# 指定查询的字段
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2018-03-22 06:38:47,131 INFO sqlalchemy.engine.base.Engine SELECT t_user.name AS t_user_name, t_user.fullname AS t_user_fullname 
FROM t_user
2018-03-22 06:38:47,135 INFO sqlalchemy.engine.base.Engine {}
ed Ed Jones
john John Doe
mike Micheal Jackson
sofie Sofie Moore


In [19]:
# 用字段名获取结果集行中的列
for row in session.query(User.name, User.fullname):
    print(row.name, row.fullname)
# 也可以像普通tuple一样，用序号获取结果集中的列
for row in session.query(User.name, User.fullname):
    print(row[0], row[1])

2018-03-22 06:38:47,154 INFO sqlalchemy.engine.base.Engine SELECT t_user.name AS t_user_name, t_user.fullname AS t_user_fullname 
FROM t_user
2018-03-22 06:38:47,157 INFO sqlalchemy.engine.base.Engine {}
ed Ed Jones
john John Doe
mike Micheal Jackson
sofie Sofie Moore
2018-03-22 06:38:47,166 INFO sqlalchemy.engine.base.Engine SELECT t_user.name AS t_user_name, t_user.fullname AS t_user_fullname 
FROM t_user
2018-03-22 06:38:47,167 INFO sqlalchemy.engine.base.Engine {}
ed Ed Jones
john John Doe
mike Micheal Jackson
sofie Sofie Moore


In [20]:
# 使用label()给字段起别名
for row in session.query(User.name.label('name_alias')):
    print(row.name_alias)

2018-03-22 06:38:47,187 INFO sqlalchemy.engine.base.Engine SELECT t_user.name AS name_alias 
FROM t_user
2018-03-22 06:38:47,190 INFO sqlalchemy.engine.base.Engine {}
ed
john
mike
sofie


In [21]:
# 使用表别名
from sqlalchemy.orm import aliased
ua = aliased(User, name='ua')
for row in session.query(ua, ua.name).all():
    print(row.ua)

2018-03-22 06:38:47,207 INFO sqlalchemy.engine.base.Engine SELECT ua.id AS ua_id, ua.name AS ua_name, ua.fullname AS ua_fullname, ua.password AS ua_password 
FROM t_user AS ua
2018-03-22 06:38:47,212 INFO sqlalchemy.engine.base.Engine {}
<User (id=1, name='ed', fullname='Ed Jones', password='654321')
<User (id=2, name='john', fullname='John Doe', password='xxxxxx')
<User (id=3, name='mike', fullname='Micheal Jackson', password='yyyyyy')
<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz')


In [22]:
# 使用Python的slicing对结果集进行limit输出
for row in session.query(User).order_by(desc(User.name))[1:3]:
    print(row.name, row.password)

2018-03-22 06:38:47,227 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user ORDER BY t_user.name DESC 
 LIMIT %(param_1)s, %(param_2)s
2018-03-22 06:38:47,230 INFO sqlalchemy.engine.base.Engine {'param_1': 1, 'param_2': 2}
mike yyyyyy
john xxxxxx


In [23]:
# 使用运算符
for row in session.query((User.name+','+User.password).label('title')):
    print(row.title)

2018-03-22 06:38:47,251 INFO sqlalchemy.engine.base.Engine SELECT concat(concat(t_user.name, %(name_1)s), t_user.password) AS title 
FROM t_user
2018-03-22 06:38:47,255 INFO sqlalchemy.engine.base.Engine {'name_1': ','}
ed,654321
john,xxxxxx
mike,yyyyyy
sofie,zzzzzz


In [24]:
# 使用filter_by()过滤结果
for row in session.query(User).filter_by(name='ed'):
    print(row.fullname)

2018-03-22 06:38:47,268 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.name = %(name_1)s
2018-03-22 06:38:47,273 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed'}
Ed Jones


In [25]:
# 更通用的，使用filter()过滤结果，注意条件表达的不一样
for row in session.query(User).filter(User.name.like('%e%')):
    print(row.fullname)
# like条件是否区分大小写取决于底层DB，因此如果确定需要不区分大小写，应该使用ilike()函数，注意两条SQL语句的不同
for row in session.query(User).filter(User.fullname.ilike('%j%')):
    print(row.fullname)

2018-03-22 06:38:47,296 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.name LIKE %(name_1)s
2018-03-22 06:38:47,298 INFO sqlalchemy.engine.base.Engine {'name_1': '%e%'}
Ed Jones
Micheal Jackson
Sofie Moore
2018-03-22 06:38:47,303 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE lower(t_user.fullname) LIKE lower(%(fullname_1)s)
2018-03-22 06:38:47,305 INFO sqlalchemy.engine.base.Engine {'fullname_1': '%j%'}
Ed Jones
John Doe
Micheal Jackson


In [26]:
# query对象的函数通常都是返回另一个query对象，因此可以继续使用filter()函数过滤结果，等同于SQL中的AND
for row in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(row)
# 下面的语句实现同样的效果
from sqlalchemy.sql import and_
for row in session.query(User).filter(and_(User.name=='ed', User.fullname=='Ed Jones')):
    print(row)

2018-03-22 06:38:47,324 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.name = %(name_1)s AND t_user.fullname = %(fullname_1)s
2018-03-22 06:38:47,327 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed', 'fullname_1': 'Ed Jones'}
<User (id=1, name='ed', fullname='Ed Jones', password='654321')
2018-03-22 06:38:47,331 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.name = %(name_1)s AND t_user.fullname = %(fullname_1)s
2018-03-22 06:38:47,334 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed', 'fullname_1': 'Ed Jones'}
<User (id=1, name='ed', fullname='Ed Jones', password='654321')


In [27]:
from sqlalchemy.orm.query import Query
query = Query(User)
# IN条件
print(query.filter(User.name.in_(['ed', 'sofie'])))
# NOT IN条件
print(query.filter(~User.name.in_(['ed', 'sofie'])))
# IS NULL
print(query.filter(User.fullname == None))
print(query.filter(User.fullname.is_(None)))
# IS NOT NULL
print(query.filter(User.fullname != None))
print(query.filter(User.fullname.isnot(None)))
# MATCH 与底层数据库实现相关
print(query.filter(User.name.match('so')))

SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.name IN (:name_1, :name_2)
SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.name NOT IN (:name_1, :name_2)
SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.fullname IS NULL
SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.fullname IS NULL
SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.fullname IS NOT NULL
SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.

#### 返回Lists和标量

In [28]:
# all() 返回全部结果集为一个List
q = session.query(User)
q.all()

2018-03-22 06:38:47,381 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user
2018-03-22 06:38:47,386 INFO sqlalchemy.engine.base.Engine {}


[<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
 <User (id=2, name='john', fullname='John Doe', password='xxxxxx'),
 <User (id=3, name='mike', fullname='Micheal Jackson', password='yyyyyy'),
 <User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz')]

In [29]:
# first()使用limit查询结果第一行并返回为标量
q.first()

2018-03-22 06:38:47,401 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
 LIMIT %(param_1)s
2018-03-22 06:38:47,405 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


<User (id=1, name='ed', fullname='Ed Jones', password='654321')

In [30]:
# 使用one()查询唯一的结果，找到多行抛出MultipleResultsFound
q.one()

2018-03-22 06:38:47,422 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user
2018-03-22 06:38:47,426 INFO sqlalchemy.engine.base.Engine {}


MultipleResultsFound: Multiple rows were found for one()

In [31]:
# 没有找到抛出NoResultFound
session.query(User).filter(User.id==100).one()

2018-03-22 06:38:53,377 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.id = %(id_1)s
2018-03-22 06:38:53,379 INFO sqlalchemy.engine.base.Engine {'id_1': 100}


NoResultFound: No row was found for one()

In [32]:
# 成功找到一条记录，返回标量
session.query(User).filter(User.id==1).one()

2018-03-22 06:38:53,896 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.id = %(id_1)s
2018-03-22 06:38:53,898 INFO sqlalchemy.engine.base.Engine {'id_1': 1}


<User (id=1, name='ed', fullname='Ed Jones', password='654321')

In [33]:
# one_or_none()类似于one()，区别在于找不到记录时，不会抛出NoResultFound，而是返回None
session.query(User).filter(User.id==100).one_or_none()

2018-03-22 06:38:54,671 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.id = %(id_1)s
2018-03-22 06:38:54,674 INFO sqlalchemy.engine.base.Engine {'id_1': 100}


### 连表查询

In [34]:
# 定义Address实体类
class Address(Base):
    __tablename__ = 't_address'
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer)
    email = Column(String(200))
    
    def __repr__(self):
        return "<Address (id=%d, user_id=%d, email='%s')>" %(self.id, self.user_id, self.email)

In [35]:
# 创建t_address表
Base.metadata.create_all(engine)

2018-03-22 06:38:56,631 INFO sqlalchemy.engine.base.Engine DESCRIBE `t_user`
2018-03-22 06:38:56,632 INFO sqlalchemy.engine.base.Engine {}
2018-03-22 06:38:56,647 INFO sqlalchemy.engine.base.Engine DESCRIBE `t_address`
2018-03-22 06:38:56,648 INFO sqlalchemy.engine.base.Engine {}
2018-03-22 06:38:56,653 INFO sqlalchemy.engine.base.Engine ROLLBACK
2018-03-22 06:38:56,658 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE t_address (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	user_id INTEGER, 
	email VARCHAR(200), 
	PRIMARY KEY (id)
)


2018-03-22 06:38:56,660 INFO sqlalchemy.engine.base.Engine {}
2018-03-22 06:38:56,695 INFO sqlalchemy.engine.base.Engine COMMIT


In [36]:
# 添加样例数据
session.add_all([
    Address(user_id=1, email='ed@msn.com'),
    Address(user_id=1, email='ed.jones@yahoo.com'),
    Address(user_id=4, email='sofie@gmail.com'),
    Address(user_id=4, email='sofie@qq.com'),
    Address(user_id=4, email='sofie.moore@example.com'),
])
session.commit()

2018-03-22 06:38:57,220 INFO sqlalchemy.engine.base.Engine INSERT INTO t_address (user_id, email) VALUES (%(user_id)s, %(email)s)
2018-03-22 06:38:57,222 INFO sqlalchemy.engine.base.Engine {'user_id': 1, 'email': 'ed@msn.com'}
2018-03-22 06:38:57,226 INFO sqlalchemy.engine.base.Engine INSERT INTO t_address (user_id, email) VALUES (%(user_id)s, %(email)s)
2018-03-22 06:38:57,227 INFO sqlalchemy.engine.base.Engine {'user_id': 1, 'email': 'ed.jones@yahoo.com'}
2018-03-22 06:38:57,230 INFO sqlalchemy.engine.base.Engine INSERT INTO t_address (user_id, email) VALUES (%(user_id)s, %(email)s)
2018-03-22 06:38:57,232 INFO sqlalchemy.engine.base.Engine {'user_id': 4, 'email': 'sofie@gmail.com'}
2018-03-22 06:38:57,235 INFO sqlalchemy.engine.base.Engine INSERT INTO t_address (user_id, email) VALUES (%(user_id)s, %(email)s)
2018-03-22 06:38:57,237 INFO sqlalchemy.engine.base.Engine {'user_id': 4, 'email': 'sofie@qq.com'}
2018-03-22 06:38:57,240 INFO sqlalchemy.engine.base.Engine INSERT INTO t_addr

In [37]:
# 笛卡尔乘积
session.query(User, Address).all()

2018-03-22 06:38:57,991 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-22 06:38:57,994 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password, t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_user, t_address
2018-03-22 06:38:57,996 INFO sqlalchemy.engine.base.Engine {}


[(<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=2, name='john', fullname='John Doe', password='xxxxxx'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=3, name='mike', fullname='Micheal Jackson', password='yyyyyy'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>),
 (<User (id=2, name='john', fullname='John Doe', password='xxxxxx'),
  <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>),
 (<User (id=3, name='mike', fullname='Micheal Jackson', password='yyyyyy'),
  <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=2, user_id=1, email='ed.jon

In [38]:
# join()连接
session.query(User, Address).join(Address, User.id==Address.user_id).all()

2018-03-22 06:38:58,432 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password, t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_user INNER JOIN t_address ON t_user.id = t_address.user_id
2018-03-22 06:38:58,435 INFO sqlalchemy.engine.base.Engine {}


[(<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=3, user_id=4, email='sofie@gmail.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=4, user_id=4, email='sofie@qq.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=5, user_id=4, email='sofie.moore@example.com')>)]

In [39]:
# 左连接outerjoin()
print(session.query(User.fullname, Address.email).outerjoin(Address, User.id==Address.user_id).all())
print(session.query(User.fullname, Address.email).join(Address, User.id==Address.user_id, isouter=True).all())

2018-03-22 06:38:59,167 INFO sqlalchemy.engine.base.Engine SELECT t_user.fullname AS t_user_fullname, t_address.email AS t_address_email 
FROM t_user LEFT OUTER JOIN t_address ON t_user.id = t_address.user_id
2018-03-22 06:38:59,170 INFO sqlalchemy.engine.base.Engine {}
[('Ed Jones', 'ed@msn.com'), ('Ed Jones', 'ed.jones@yahoo.com'), ('Sofie Moore', 'sofie@gmail.com'), ('Sofie Moore', 'sofie@qq.com'), ('Sofie Moore', 'sofie.moore@example.com'), ('John Doe', None), ('Micheal Jackson', None)]
2018-03-22 06:38:59,176 INFO sqlalchemy.engine.base.Engine SELECT t_user.fullname AS t_user_fullname, t_address.email AS t_address_email 
FROM t_user LEFT OUTER JOIN t_address ON t_user.id = t_address.user_id
2018-03-22 06:38:59,177 INFO sqlalchemy.engine.base.Engine {}
[('Ed Jones', 'ed@msn.com'), ('Ed Jones', 'ed.jones@yahoo.com'), ('Sofie Moore', 'sofie@gmail.com'), ('Sofie Moore', 'sofie@qq.com'), ('Sofie Moore', 'sofie.moore@example.com'), ('John Doe', None), ('Micheal Jackson', None)]


### 使用SQL语句

In [40]:
# 用text()函数传入SQL语句
from sqlalchemy import text
for row in session.query(User).filter(text('id<3')).order_by(text('name')):
    print(row.fullname)

2018-03-22 06:39:01,149 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE id<3 ORDER BY name
2018-03-22 06:39:01,152 INFO sqlalchemy.engine.base.Engine {}
Ed Jones
John Doe


In [41]:
# 使用from_statement和text传入整个SQL语句执行
session.query(User).from_statement(text('select * from t_user where name=:name')).params(name='ed').all()

2018-03-22 06:39:02,106 INFO sqlalchemy.engine.base.Engine select * from t_user where name=%(name)s
2018-03-22 06:39:02,108 INFO sqlalchemy.engine.base.Engine {'name': 'ed'}


[<User (id=1, name='ed', fullname='Ed Jones', password='654321')]

In [42]:
# 使用SQL语句连表查询
session.query(User.fullname, Address.email).from_statement(text(
    'select a.fullname, b.email from t_user a join t_address b '
    'on a.id=b.user_id')
).all()

2018-03-22 06:39:02,665 INFO sqlalchemy.engine.base.Engine select a.fullname, b.email from t_user a join t_address b on a.id=b.user_id
2018-03-22 06:39:02,668 INFO sqlalchemy.engine.base.Engine {}


[('Ed Jones', 'ed@msn.com'),
 ('Ed Jones', 'ed.jones@yahoo.com'),
 ('Sofie Moore', 'sofie@gmail.com'),
 ('Sofie Moore', 'sofie@qq.com'),
 ('Sofie Moore', 'sofie.moore@example.com')]

### 计数

In [43]:
# 使用count()对结果集计数
session.query(User).filter(User.name.like('%e%')).count()

2018-03-22 06:39:04,103 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.name LIKE %(name_1)s) AS anon_1
2018-03-22 06:39:04,108 INFO sqlalchemy.engine.base.Engine {'name_1': '%e%'}


3

In [44]:
# 使用func模块的SQL count计数
from sqlalchemy import func
session.query(User.name, func.count(User.name)).group_by(User.name).all()

2018-03-22 06:39:06,082 INFO sqlalchemy.engine.base.Engine SELECT t_user.name AS t_user_name, count(t_user.name) AS count_1 
FROM t_user GROUP BY t_user.name
2018-03-22 06:39:06,084 INFO sqlalchemy.engine.base.Engine {}


[('ed', 1), ('john', 1), ('mike', 1), ('sofie', 1)]

In [45]:
# 全表计数
session.query(func.count(User.id)).scalar()

2018-03-22 06:39:07,002 INFO sqlalchemy.engine.base.Engine SELECT count(t_user.id) AS count_1 
FROM t_user
2018-03-22 06:39:07,004 INFO sqlalchemy.engine.base.Engine {}


4

### 使用relationship

In [46]:
# 创建两张表的relationship，在primaryjoin中设定foreign关系，并不会设定数据库表之间的外键约束
from sqlalchemy.orm import relationship
Address.user = relationship('User', primaryjoin='User.id==foreign(Address.user_id)')
User.address = relationship('Address', primaryjoin='foreign(User.id)==Address.user_id', uselist=True)

In [47]:
# 查看ed的所有电子邮件地址
ed_user.address

2018-03-22 06:39:24,617 INFO sqlalchemy.engine.base.Engine SELECT t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_address 
WHERE %(param_1)s = t_address.user_id
2018-03-22 06:39:24,620 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[<Address (id=1, user_id=1, email='ed@msn.com')>,
 <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>]

In [48]:
# 添加一条地址给ed
session.add(Address(user_id=1, email='xx@yy.zz'))
session.commit()

2018-03-22 06:39:25,244 INFO sqlalchemy.engine.base.Engine INSERT INTO t_address (user_id, email) VALUES (%(user_id)s, %(email)s)
2018-03-22 06:39:25,249 INFO sqlalchemy.engine.base.Engine {'user_id': 1, 'email': 'xx@yy.zz'}
2018-03-22 06:39:25,253 INFO sqlalchemy.engine.base.Engine COMMIT


In [49]:
# 对应的address属性相应发生改变
ed_user.address

2018-03-22 06:39:26,271 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-22 06:39:26,277 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.id = %(param_1)s
2018-03-22 06:39:26,279 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2018-03-22 06:39:26,282 INFO sqlalchemy.engine.base.Engine SELECT t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_address 
WHERE %(param_1)s = t_address.user_id
2018-03-22 06:39:26,284 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[<Address (id=1, user_id=1, email='ed@msn.com')>,
 <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>,
 <Address (id=6, user_id=1, email='xx@yy.zz')>]

In [50]:
# 修改该条地址对应的user
session.rollback()
lemail = session.query(Address).filter(Address.id==6).one()
lemail.user_id=2
session.commit()

2018-03-22 06:39:28,333 INFO sqlalchemy.engine.base.Engine ROLLBACK
2018-03-22 06:39:28,337 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-22 06:39:28,339 INFO sqlalchemy.engine.base.Engine SELECT t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_address 
WHERE t_address.id = %(id_1)s
2018-03-22 06:39:28,341 INFO sqlalchemy.engine.base.Engine {'id_1': 6}
2018-03-22 06:39:28,346 INFO sqlalchemy.engine.base.Engine UPDATE t_address SET user_id=%(user_id)s WHERE t_address.id = %(t_address_id)s
2018-03-22 06:39:28,347 INFO sqlalchemy.engine.base.Engine {'user_id': 2, 't_address_id': 6}
2018-03-22 06:39:28,350 INFO sqlalchemy.engine.base.Engine COMMIT


In [51]:
# address属性相应发生改变
ed_user.address

2018-03-22 06:39:29,290 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-22 06:39:29,293 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.id = %(param_1)s
2018-03-22 06:39:29,294 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2018-03-22 06:39:29,299 INFO sqlalchemy.engine.base.Engine SELECT t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_address 
WHERE %(param_1)s = t_address.user_id
2018-03-22 06:39:29,301 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[<Address (id=1, user_id=1, email='ed@msn.com')>,
 <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>]

In [52]:
ed_user.address[1].user

<User (id=1, name='ed', fullname='Ed Jones', password='654321')

#### JOIN高级用法

In [53]:
# join()函数中指定连接条件
session.query(User, Address).join(Address, User.id==Address.user_id).all()

2018-03-22 06:39:32,168 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password, t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_user INNER JOIN t_address ON t_user.id = t_address.user_id
2018-03-22 06:39:32,171 INFO sqlalchemy.engine.base.Engine {}


[(<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>),
 (<User (id=2, name='john', fullname='John Doe', password='xxxxxx'),
  <Address (id=6, user_id=2, email='xx@yy.zz')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=3, user_id=4, email='sofie@gmail.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=4, user_id=4, email='sofie@qq.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=5, user_id=4, email='sofie.moore@example.com')>)]

In [54]:
# join()中使用relationship
session.query(User, Address).join(User.address).all()

2018-03-22 06:39:33,490 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password, t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_user INNER JOIN t_address ON t_user.id = t_address.user_id
2018-03-22 06:39:33,493 INFO sqlalchemy.engine.base.Engine {}


[(<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>),
 (<User (id=2, name='john', fullname='John Doe', password='xxxxxx'),
  <Address (id=6, user_id=2, email='xx@yy.zz')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=3, user_id=4, email='sofie@gmail.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=4, user_id=4, email='sofie@qq.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=5, user_id=4, email='sofie.moore@example.com')>)]

In [55]:
# join()中使用relationship的字符串条件进行连接
session.query(User, Address).join('address').all()

2018-03-22 06:39:34,850 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password, t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_user INNER JOIN t_address ON t_user.id = t_address.user_id
2018-03-22 06:39:34,852 INFO sqlalchemy.engine.base.Engine {}


[(<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>),
 (<User (id=2, name='john', fullname='John Doe', password='xxxxxx'),
  <Address (id=6, user_id=2, email='xx@yy.zz')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=3, user_id=4, email='sofie@gmail.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=4, user_id=4, email='sofie@qq.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=5, user_id=4, email='sofie.moore@example.com')>)]

#### 使用子查询

In [56]:
# 查询每个用户拥有的email数量
s = session.query(Address.user_id, func.count(Address.id).label('address_count')).group_by(Address.user_id).subquery()
session.query(User, s.c.address_count).outerjoin(s, User.id==s.c.user_id).all()

2018-03-22 06:39:36,768 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password, anon_1.address_count AS anon_1_address_count 
FROM t_user LEFT OUTER JOIN (SELECT t_address.user_id AS user_id, count(t_address.id) AS address_count 
FROM t_address GROUP BY t_address.user_id) AS anon_1 ON t_user.id = anon_1.user_id
2018-03-22 06:39:36,770 INFO sqlalchemy.engine.base.Engine {}


[(<User (id=1, name='ed', fullname='Ed Jones', password='654321'), 2),
 (<User (id=2, name='john', fullname='John Doe', password='xxxxxx'), 1),
 (<User (id=3, name='mike', fullname='Micheal Jackson', password='yyyyyy'),
  None),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'), 3)]

In [57]:
# 将子查询的结果放入实体对象中，需要使用aliased()函数将子查询与实体类关联在一起
s = session.query(Address).filter(Address.email!='sofie@qq.com').subquery()
adt = aliased(Address, s)
session.query(User, adt).join(adt, User.address).all()

2018-03-22 06:39:38,788 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password, anon_1.id AS anon_1_id, anon_1.user_id AS anon_1_user_id, anon_1.email AS anon_1_email 
FROM t_user INNER JOIN (SELECT t_address.id AS id, t_address.user_id AS user_id, t_address.email AS email 
FROM t_address 
WHERE t_address.email != %(email_1)s) AS anon_1 ON t_user.id = anon_1.user_id
2018-03-22 06:39:38,791 INFO sqlalchemy.engine.base.Engine {'email_1': 'sofie@qq.com'}


[(<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>),
 (<User (id=1, name='ed', fullname='Ed Jones', password='654321'),
  <Address (id=1, user_id=1, email='ed@msn.com')>),
 (<User (id=2, name='john', fullname='John Doe', password='xxxxxx'),
  <Address (id=6, user_id=2, email='xx@yy.zz')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=5, user_id=4, email='sofie.moore@example.com')>),
 (<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz'),
  <Address (id=3, user_id=4, email='sofie@gmail.com')>)]

#### 使用exists

In [58]:
# exists条件
from sqlalchemy.sql import exists
s = exists().where(Address.user_id == User.id)
session.query(User.fullname).filter(s).all()

2018-03-22 06:39:40,924 INFO sqlalchemy.engine.base.Engine SELECT t_user.fullname AS t_user_fullname 
FROM t_user 
WHERE EXISTS (SELECT * 
FROM t_address 
WHERE t_address.user_id = t_user.id)
2018-03-22 06:39:40,926 INFO sqlalchemy.engine.base.Engine {}


[('Ed Jones'), ('John Doe'), ('Sofie Moore')]

In [59]:
# 使用relationship的any()函数也可以达到一样的效果
session.query(User.fullname).filter(User.address.any()).all()

2018-03-22 06:39:41,476 INFO sqlalchemy.engine.base.Engine SELECT t_user.fullname AS t_user_fullname 
FROM t_user 
WHERE EXISTS (SELECT 1 
FROM t_address 
WHERE t_user.id = t_address.user_id)
2018-03-22 06:39:41,479 INFO sqlalchemy.engine.base.Engine {}


[('Ed Jones'), ('John Doe'), ('Sofie Moore')]

In [60]:
# any()函数中还可以加入条件
session.query(User.fullname).filter(User.address.any(Address.email.like('%gmail%'))).all()

2018-03-22 06:39:42,176 INFO sqlalchemy.engine.base.Engine SELECT t_user.fullname AS t_user_fullname 
FROM t_user 
WHERE EXISTS (SELECT 1 
FROM t_address 
WHERE t_user.id = t_address.user_id AND t_address.email LIKE %(email_1)s)
2018-03-22 06:39:42,178 INFO sqlalchemy.engine.base.Engine {'email_1': '%gmail%'}


[('Sofie Moore')]

In [61]:
# relationship的has()函数与any()相似，但是是用在多对一关系中
session.query(Address.email).filter(Address.user.has(User.name=='ed')).all()

2018-03-22 06:39:43,022 INFO sqlalchemy.engine.base.Engine SELECT t_address.email AS t_address_email 
FROM t_address 
WHERE EXISTS (SELECT 1 
FROM t_user 
WHERE t_user.id = t_address.user_id AND t_user.name = %(name_1)s)
2018-03-22 06:39:43,026 INFO sqlalchemy.engine.base.Engine {'name_1': 'ed'}


[('ed@msn.com'), ('ed.jones@yahoo.com')]

#### relationship的运算符

In [62]:
# __eq()__ == 多对一关系中比较相等
session.query(Address).filter(Address.user == ed_user).all()

2018-03-22 06:39:44,273 INFO sqlalchemy.engine.base.Engine SELECT t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_address 
WHERE %(param_1)s = t_address.user_id
2018-03-22 06:39:44,275 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[<Address (id=1, user_id=1, email='ed@msn.com')>,
 <Address (id=2, user_id=1, email='ed.jones@yahoo.com')>]

In [63]:
# __ne()__ != 多对一关系中比较不等
session.query(Address).filter(Address.user != ed_user).all()

2018-03-22 06:39:44,942 INFO sqlalchemy.engine.base.Engine SELECT t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_address 
WHERE t_address.user_id != %(user_id_1)s OR t_address.user_id IS NULL
2018-03-22 06:39:44,945 INFO sqlalchemy.engine.base.Engine {'user_id_1': 1}


[<Address (id=3, user_id=4, email='sofie@gmail.com')>,
 <Address (id=4, user_id=4, email='sofie@qq.com')>,
 <Address (id=5, user_id=4, email='sofie.moore@example.com')>,
 <Address (id=6, user_id=2, email='xx@yy.zz')>]

In [64]:
# 空值比较
session.query(Address).filter(Address.user == None).all()

2018-03-22 06:39:45,464 INFO sqlalchemy.engine.base.Engine SELECT t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_address 
WHERE t_address.user_id IS NULL
2018-03-22 06:39:45,466 INFO sqlalchemy.engine.base.Engine {}


[]

In [65]:
# contains()，一对多关系中比较是否包含
gmail = session.query(Address).filter(Address.email.like('%gmail%')).one()
session.query(User).filter(User.address.contains(gmail)).all()

2018-03-22 06:39:46,098 INFO sqlalchemy.engine.base.Engine SELECT t_address.id AS t_address_id, t_address.user_id AS t_address_user_id, t_address.email AS t_address_email 
FROM t_address 
WHERE t_address.email LIKE %(email_1)s
2018-03-22 06:39:46,101 INFO sqlalchemy.engine.base.Engine {'email_1': '%gmail%'}
2018-03-22 06:39:46,107 INFO sqlalchemy.engine.base.Engine SELECT t_user.id AS t_user_id, t_user.name AS t_user_name, t_user.fullname AS t_user_fullname, t_user.password AS t_user_password 
FROM t_user 
WHERE t_user.id = %(param_1)s
2018-03-22 06:39:46,109 INFO sqlalchemy.engine.base.Engine {'param_1': 4}


[<User (id=4, name='sofie', fullname='Sofie Moore', password='zzzzzz')]